In [1]:
import pandas as pd
import joblib
import numpy as np

def predict_new_user(user_data):
    """
    Calculates features from chatbot data and predicts an A.D.I.T.I. score.

    Args:
        user_data (dict): A dictionary containing the new user's raw answers from the chatbot.

    Returns:
        int: The final calculated A.D.I.T.I. credit score.
    """
    try:
        # --- 1. Calculate the 5 Core Features from Chatbot Data ---
        # This logic now mirrors your final JavaScript prototype exactly.

        # Feature 1: Final Utility Score
        on_time_bills = user_data.get('utility_on_time', 0)
        late_bills = 12 - on_time_bills
        final_utility_score = max(0, 10 - (late_bills * 1.5))

        # Feature 2: Recharge Consistency Score (using SIM age as a proxy, like in the JS)
        sim_age_months = user_data.get('sim_age_months', 0)
        recharge_consistency_score = 0.9 if sim_age_months > 12 else 0.6

        # Feature 3: Digital Spend Index
        min_spend, max_spend = 0, 50000
        avg_monthly_spend = user_data.get('avg_monthly_spend', 0)
        digital_spend_index = (avg_monthly_spend - min_spend) / (max_spend - min_spend) * 100

        # Feature 4: Stability Score
        stability_score = 0
        if sim_age_months >= 24:
            stability_score = 10
        elif sim_age_months >= 12:
            stability_score = 6
        else:
            stability_score = 2
        # Add bonus for good bill payment as a proxy for address stability
        if final_utility_score > 8:
            stability_score = min(10, stability_score + 2)

        # Feature 5: Psychometric Score
        # Averaging the scores from the three psychometric questions
        psycho_scores = [
            user_data.get('loan_goal_value', 0),
            user_data.get('bonus_instinct_value', 0),
            user_data.get('monthly_savings_value', 0)
        ]
        psychometric_score = np.mean(psycho_scores)

        # --- 2. Load the Trained Model ---
        model = joblib.load('C:\\Users\\Bhuvana\\Desktop\\ADITI-Final-Submission\\backend\\aditi_model.pkl')

        # --- 3. Create a DataFrame for the new user ---
        feature_names = [
            'final_utility_score', 'recharge_consistency_score', 'digital_spend_index',
            'stability_score', 'psychometric_score'
        ]
        new_user_df = pd.DataFrame([[
            final_utility_score,
            recharge_consistency_score,
            digital_spend_index,
            stability_score,
            psychometric_score
        ]], columns=feature_names)

        # --- 4. Predict the Probability of Default (PD) ---
        probability_of_default = model.predict_proba(new_user_df)[0][1]

        # --- 5. Calculate the Final A.D.I.T.I. Score ---
        final_score = 300 + (600 * (1 - probability_of_default))

        return int(round(final_score))

    except FileNotFoundError:
        return "Error: 'aditi_model.pkl' not found. Please run train_model.py first."
    except Exception as e:
        return f"An error occurred: {e}"

# --- Example Usage ---
if __name__ == "__main__":
    # Simulate a new user's information based on the CHATBOT questions

    # --- Profile 1: A Good Applicant ---
    good_user_data = {
        "loan_goal_value": 1.0,           # Chose "To start or expand a business"
        "utility_on_time": 12,            # Chose "All 12 bills"
        "sim_age_months": 24,             # Chose "More than 2 years"
        "avg_monthly_spend": 17500,       # Chose "₹10,000 to ₹25,000"
        "bonus_instinct_value": 1.0,      # Chose "Save or invest most of it"
        "monthly_savings_value": 0.7      # Chose "10% to 25%"
    }

    # --- Profile 2: A Risky Applicant ---
    risky_user_data = {
        "loan_goal_value": 0.1,           # Chose "To buy a luxury item"
        "utility_on_time": 6,             # Chose "7 bills or less"
        "sim_age_months": 6,              # Chose "Less than 1 year"
        "avg_monthly_spend": 2500,        # Chose "Less than ₹5,000"
        "bonus_instinct_value": 0.2,      # Chose "Spend it on something I've wanted"
        "monthly_savings_value": 0.1      # Chose "Nothing, it's very tight"
    }

    print("--- Predicting New User Scores (Aligned with Prototype Logic) ---")

    good_user_score = predict_new_user(good_user_data)
    print(f"Predicted A.D.I.T.I. Score for the Good User: {good_user_score}")

    risky_user_score = predict_new_user(risky_user_data)
    print(f"Predicted A.D.I.T.I. Score for the Risky User: {risky_user_score}")

--- Predicting New User Scores (Aligned with Prototype Logic) ---
Predicted A.D.I.T.I. Score for the Good User: 899
Predicted A.D.I.T.I. Score for the Risky User: 304
